In [ ]:
import plotly.graph_objects as go
import re

# Data from the provided JSON
categories = [
    "Аренда/покупка сервера",
    "Лицензии ПО",
    "Электричество/охлаждение",
    "Администрирование",
    "Резервное копирование",
    "Миграция/развертывание VM",
    "Высокая доступность/HA"
]

# Parse costs and calculate midpoints for ranges
def parse_cost(cost_str):
    # Remove $ and commas, extract numbers
    numbers = re.findall(r'\d+(?:,\d+)*', cost_str.replace('$', ''))
    if not numbers:
        return 0
    # Convert to integers
    nums = [int(n.replace(',', '')) for n in numbers]
    # Return midpoint if range, otherwise return the value
    if len(nums) >= 2 and '-' in cost_str:
        return (nums[0] + nums[1]) / 2
    return nums[0]

dediki_costs = [
    7200,  # Server rental
    1000,  # Licenses (midpoint of 0-2000)
    2500,  # Power/cooling (midpoint of 2000-3000)
    20000,  # Administration
    2250,  # Backup (midpoint of 1500-3000)
    4000,  # VM deployment (midpoint of 3000-5000)
    5500   # HA cluster (midpoint of 3000-8000)
]

proxmox_costs = [
    4800,  # Server rental
    0,     # Licenses
    1500,  # Power/cooling (midpoint of 1200-1800)
    15000, # Administration
    500,   # Backup
    500,   # VM deployment
    400    # HA cluster
]

# Create grouped bar chart
fig = go.Figure()

fig.add_trace(go.Bar(
    name='Dediki',
    x=categories,
    y=dediki_costs,
    marker_color='#DB4545',
    text=[f'${v/1000:.1f}k' for v in dediki_costs],
    textposition='outside',
    cliponaxis=False
))

fig.add_trace(go.Bar(
    name='Proxmox',
    x=categories,
    y=proxmox_costs,
    marker_color='#1FB8CD',
    text=[f'${v/1000:.1f}k' for v in proxmox_costs],
    textposition='outside',
    cliponaxis=False
))

# Calculate totals
dediki_total = sum(dediki_costs)
proxmox_total = sum(proxmox_costs)
savings = dediki_total - proxmox_total

fig.update_layout(
    title={
        "text": f"Proxmox Saves ${savings/1000:.1f}k Annually vs Dediki<br><span style='font-size: 18px; font-weight: normal;'>Corporate service with 100 VMs | 3-year savings: ${savings*3/1000:.1f}k</span>"
    },
    xaxis_title="Категория затрат",
    yaxis_title="Стоимость ($)",
    barmode='group',
    legend=dict(
        orientation='h',
        yanchor='bottom',
        y=1.05,
        xanchor='center',
        x=0.5
    )
)

# Format y-axis with k/m abbreviations
fig.update_yaxes(tickformat='$,.0f')

# Wrap x-axis labels
fig.update_xaxes(tickangle=-45)